<a href="https://colab.research.google.com/github/mosama1994/Llama2-Prompt-Reverse-Engineered/blob/main/Reverse_Engineering_Llama2_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizer.model file included in the repository, change below path to that file

In [30]:
model_path = '/content/drive/MyDrive/Llama-2-7b-chat-hf/tokenizer.model'

In [29]:
!pip install sentencepiece --quiet

In [10]:
dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]

In [3]:
from typing import List, Literal, Optional, Tuple, TypedDict

In [6]:
Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str


class CompletionPrediction(TypedDict, total=False):
    generation: str
    tokens: List[str]  # not required
    logprobs: List[float]  # not required


class ChatPrediction(TypedDict, total=False):
    generation: Message
    tokens: List[str]  # not required
    logprobs: List[float]  # not required

# System Prompt, you can change it here

In [7]:
Dialog = List[Message]

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

In [14]:
import os
from logging import getLogger

from sentencepiece import SentencePieceProcessor

logger = getLogger()


class Tokenizer:
    def __init__(self, model_path: str):
        # reload tokenizer
        assert os.path.isfile(model_path), model_path
        self.sp_model = SentencePieceProcessor(model_file=model_path)
        logger.info(f"Reloaded SentencePiece model from {model_path}")

        # BOS / EOS token IDs
        self.n_words: int = self.sp_model.vocab_size()
        self.bos_id: int = self.sp_model.bos_id()
        self.eos_id: int = self.sp_model.eos_id()
        self.pad_id: int = self.sp_model.pad_id()
        logger.info(
            f"#words: {self.n_words} - BOS ID: {self.bos_id} - EOS ID: {self.eos_id}"
        )
        assert self.sp_model.vocab_size() == self.sp_model.get_piece_size()

    def encode(self, s: str, bos: bool, eos: bool) -> List[int]:
        assert type(s) is str
        t = self.sp_model.encode(s)
        if bos:
            t = [self.bos_id] + t
        if eos:
            t = t + [self.eos_id]
        return t

    def decode(self, t: List[int]) -> str:
        return self.sp_model.decode(t)

In [8]:
def chat_completion(
        dialogs: List[Dialog],
        temperature: float = 0.6,
        top_p: float = 0.9,
        max_gen_len: Optional[int] = None,
        logprobs: bool = False
    ) -> List[ChatPrediction]:

        inter_tokens = []
        prompt_tokens = []
        for dialog in dialogs:
            if dialog[0]["role"] != "system":
                dialog = [
                    {
                        "role": "system",
                        "content": DEFAULT_SYSTEM_PROMPT,
                    }
                ] + dialog
            dialog = [
                {
                    "role": dialog[1]["role"],
                    "content": B_SYS
                    + dialog[0]["content"]
                    + E_SYS
                    + dialog[1]["content"],
                }
            ] + dialog[2:]
            assert all([msg["role"] == "user" for msg in dialog[::2]]) and all(
                [msg["role"] == "assistant" for msg in dialog[1::2]]
            ), (
                "model only supports 'system', 'user' and 'assistant' roles, "
                "starting with 'system', then 'user' and alternating (u/a/u/a/u...)"
            )

            dialog_tokens: List[int] = sum(
                [
                    tokenizer.encode(
                        f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} ",
                        bos=True,
                        eos=True,
                    )
                    for prompt, answer in zip(
                        dialog[::2],
                        dialog[1::2],
                    )
                ],
                [],
            )
            assert (
                dialog[-1]["role"] == "user"
            ), f"Last message must be from user, got {dialog[-1]['role']}"
            dialog_tokens += tokenizer.encode(
                f"{B_INST} {(dialog[-1]['content']).strip()} {E_INST}",
                bos=True,
                eos=False,
            )
            prompt_tokens.append(dialog_tokens)

        return prompt_tokens

# Change this tokenizer path to the path of the tokenizer.model file

In [15]:
tokenizer = Tokenizer(model_path)

In [16]:
results = chat_completion(dialogs)

# Change this model path to the path of the tokenizer.model file

## Checking the BOS and EOS symbols in Sentence Piece

In [17]:
from sentencepiece import SentencePieceProcessor
sp_model = SentencePieceProcessor(model_file=model_path)

bos_symbol = sp_model.id_to_piece(sp_model.bos_id())
eos_symbol = sp_model.id_to_piece(sp_model.eos_id())
print(f"End of sequence (EOS) symbol is: {bos_symbol}")
print(f"End of sequence (EOS) symbol is: {eos_symbol}")

End of sequence (EOS) symbol is: <s>
End of sequence (EOS) symbol is: </s>


# Defining the decoder function to include BOS and EOS tokens

In [33]:
# We will make a nested list, [[token index, 1 for bos and 2 for eos], [], ......]
def decode_prompt(results):
  bos_eos_list = []

  for i in range(0,len(results)):
    if results[i] == 1:
      bos_eos_list.append([i,1])
    if results[i] == 2:
      bos_eos_list.append([i,2])

  full_text = ""

  for i in range(0,len(bos_eos_list)):
    if i == (len(bos_eos_list) - 1):
      decoded_text = tokenizer.decode(results[bos_eos_list[i][0]:])
      decoded_text = "<s>" + decoded_text
    else:
      decoded_text = tokenizer.decode(results[bos_eos_list[i][0]:bos_eos_list[i+1][0]])
      if i == 0:
        decoded_text = "<s>" + decoded_text
      else:
        decoded_text = "</s>" + decoded_text

    full_text += decoded_text

  return full_text

# You can change the system message in the prompt defined toward the top

In [34]:
for i in list(zip(dialogs, results)):
  print("Dialog with Roles defined:\n")
  print(i[0])
  print()
  print("Dialog Best Practice to send as Input to Model:\n")
  print(decode_prompt(i[1]))
  print()

Dialog with Roles defined:

[{'role': 'user', 'content': 'what is the recipe of mayonnaise?'}]

Dialog Best Practice to send as Input to Model:

<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

what is the recipe of mayonnaise? [/INST]

Dialog with Roles defined:

[{'role': 'user', 'content': 'I am going to Paris, what should I see?'}, {'role': 'assistant', 'content': "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are so

# Prompt Style Simplified

In [27]:
"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{user_message_1} [/INST]"""

'<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message_1} [/INST]'

In [28]:
"""<s>[INST] <<SYS>>
{your_system_message}
<</SYS>>

{user_message_1} [/INST] {model_reply_1}</s><s>[INST] {user_message_2} [/INST]"""

'<s>[INST] <<SYS>>\n{your_system_message}\n<</SYS>>\n\n{user_message_1} [/INST] {model_reply_1}</s><s>[INST] {user_message_2} [/INST]'